In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm_notebook
from pathlib import Path
import random
import torch.nn.init as init

In [ ]:
# TCN
!pip install pytorch-tcn
from pytorch_tcn import TCN

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

# import os

# google_drive_path = "G:/"


In [4]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

# torch and python versions ====================================================
print('torch.__version__:',torch.__version__)
print('sys.version:',sys.version)

In [6]:
DATASET_NAMES = ['Site 3']
VARIABLE_NAMES = ['Temperature', 'V', 'Sin(dir)','Cos(dir)']
UNITS_OF_MEASUREMENT = ['°C', 'm/s', '', '']
POSITIVE_VARIABLES = [1]

In [7]:
# Hyperparameters ==============================================================
LOOK_FORWARD_DAYS_LIST = [1,2,3]
DATASETS_TO_PROCESS = [0]
VARIABLES_TO_PROCESS = [1,2,3]

LOOK_BACK_DAYS = 3
TEST_FRACTION = 0.2
VALIDATION_FRACTION = 0.2
PATIENCE = 10
SHUFFLE = True
FORCE_TRAINING = False
DEBUG = False
USE_ADAM = False
USE_EARLY_STOP = True
USE_VALIDATION = True
DAYS_TO_PLOT = 3
POST_PROCESS = True
PRINT_RMSE = False

STANDARD_SCALERS = False
PLOT_DISTRIBUTIONS = False
OUTLIERS = 0.25
# ==============================================================================

if not USE_VALIDATION:
  VALIDATION_FRACTION = 0.0

MODELS = ['gru', 'tcn']

HP = {0: # Site 3
         {0:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2,
                'BATCH_SIZE' : 512,
                'LEARNING_RATE' : 0.001,
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200,
                'NUM_CHANNELS' : [32, 64],
                'KERNEL_SIZE' : 3,
                'SKIP_CONNECTIONS' : False},

          1:  { 'HIDDEN_DIM' :128,
                'N_LAYERS' : 2,
                'BATCH_SIZE' : 512,
                'LEARNING_RATE' : 0.001,
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200,
                'NUM_CHANNELS' : [32, 64],
                'KERNEL_SIZE' : 3,
                'SKIP_CONNECTIONS' : False},

          2:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2, #4
                'BATCH_SIZE' : 512, #256
                'LEARNING_RATE' : 0.001,
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200,
                'NUM_CHANNELS' : [32, 64],
                'KERNEL_SIZE' : 3,
                'SKIP_CONNECTIONS' : False},

          3:  { 'HIDDEN_DIM' : 128,
                'N_LAYERS' : 2, #4
                'BATCH_SIZE' : 512, #256
                'LEARNING_RATE' : 0.001,  #0.0001
                'TEST_PLOT_START' : [9035, 8920, 8800],
                'EPOCHS' : 200,
                'NUM_CHANNELS' : [32, 64],
                'KERNEL_SIZE' : 3,
                'SKIP_CONNECTIONS' : False}
        },
}

In [8]:
# Loop through each dataset in DATASETS_TO_PROCESS
for dataset_id in DATASETS_TO_PROCESS:
  # Access the configuration dictionary for the current dataset
  dataset_config = HP[dataset_id]

  # Loop through each variable configuration within the dataset
  for variable_id in VARIABLES_TO_PROCESS:
    # Access the 'HIDDEN_DIM' value for the current variable
    hidden_dim = dataset_config[variable_id]['HIDDEN_DIM']
  for variable_id in VARIABLES_TO_PROCESS:
    # Access the 'N_LAYERS' value for the current variable
    n_layers = dataset_config[variable_id]['N_LAYERS']

In [9]:
def process_epoch(model, data_loader, criterion, optimizer, train):
    n_layers = model.n_layers
    hidden_dim = model.hidden_dim
    batch_size = data_loader.batch_size

    if type(model) == GRUModel:
      if SHUFFLE:
        h = torch.zeros([n_layers, batch_size, hidden_dim]).to(device)
      else:
        h = model.init_hidden(batch_size)

    total_loss = 0.
    counter = 0

    for x, y, local_means in data_loader:
        counter += 1

        if type(model) == GRUModel:
          if SHUFFLE:
            h = torch.zeros([n_layers, batch_size, hidden_dim]).to(device)
          else:
            h = h.data

        model.zero_grad()

        if type(model) == GRUModel:
          y_hat, h = model(x.to(device).float(), h)
          # print('Shape H:',h.shape)
        elif type(model) == TCNModel:
          y_hat = model(x.to(device).float())

        base_loss = criterion(y_hat, y.to(device).float())

        #local_mean_loss = torch.mean(torch.abs(y_hat - local_means.to(device).float()))  #l1
        #local_mean_loss = torch.mean((y_hat - local_means.to(device).float()) ** 2) # mse
        local_mean_loss = criterion(y_hat, local_means.to(device).float())

        α = 0.5
        #loss = base_loss
        #loss = local_mean_loss
        loss = α * base_loss + (1-α) * local_mean_loss

        total_loss += loss.item()

        if train:
            loss.backward()
            optimizer.step()

        if counter%20==0:
            print("Step: {}/{}, Average Loss: {}".format(counter, len(data_loader), total_loss/counter))

    avg_loss = total_loss/len(data_loader)

    return avg_loss

def train(model_name, train_loader, validation_loader, learning_rate, positive_variable, batch_size, hidden_dim=64, num_channels = None, kernel_size = None, skip_connections = False, n_layers=2, patience=None, epochs=None):
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1

    print('Input dim.:', input_dim)
    print('Output dim.:', output_dim)

    if 'tcn' in model_name:
      print("tcn è in model_name")
      model = TCNModel(input_dim, hidden_dim, output_dim, n_layers, positive_variable, kernel_size, num_channels, skip_connections)
    else:
      model = GRUModel(input_dim, hidden_dim, output_dim, n_layers, positive_variable)

    model.to(device)

    # Loss function and optimizer
    #criterion = nn.MSELoss()
    criterion = nn.L1Loss()

    if USE_ADAM:
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    else:
      optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

    model.train()

    epoch=1
    patience_counter = 0
    min_validation_loss=None
    validation_loss=None

    train_loss_list=[]
    validation_loss_list=[]

    # Training loop ============================================================
    while True:
        print("Epoch {}".format(epoch))

        print('Training...')
        train_loss = process_epoch(model, train_loader, criterion, optimizer, train = True)

        if USE_VALIDATION:
          print('Validation...')
          validation_loss = process_epoch(model, validation_loader, criterion, optimizer, train = False)
          validation_loss_list.append(validation_loss)

        train_loss_list.append(train_loss)

        if min_validation_loss is None or validation_loss<min_validation_loss or not USE_VALIDATION:
          # Save the model
          print('Saving the model...')
          if 'gru' in model_name:
            torch.save(model, model_name)
          elif 'tcn' in model_name:
            torch.save(model.state_dict(), model_name)

          min_validation_loss = validation_loss
          patience_counter = 0
        else:
          print('Increasing patience counter')
          patience_counter +=1

        print("Epoch {} Completed, Train Loss: {}, Validation Loss: {}, Patience counter: {}".format(epoch, train_loss, validation_loss, patience_counter))

        if (patience_counter>patience and USE_EARLY_STOP) or epoch==epochs:
          break

        epoch += 1

    print('Restoring the best model...')
    if 'gru' in model_name:
      model = torch.load(model_name, weights_only=False)
    elif 'tcn' in model_name:
      model.load_state_dict(torch.load(model_name))
    return model, train_loss_list, validation_loss_list

In [10]:
import abc
from abc import ABC, abstractmethod

class Model(nn.Module, ABC):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, positive_variable, num_channels = None, drop_prob=0.0): # num_channels = None because it
        super().__init__()                                                                                                  # doesn't exist in the rnns
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.positive_variable=positive_variable

    @abstractmethod
    def forward(self, x, h = None): # h = None because it doesn't exist in the tcns
        pass

class GRUModel(Model):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, positive_variable, drop_prob=0.0):
        super().__init__(input_dim, hidden_dim, output_dim, n_layers, positive_variable, drop_prob)

        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        #self.bat_a = nn.BatchNorm1d(hidden_dim)
        #self.bat_b = nn.BatchNorm1d(hidden_dim)
        self.activation_a = nn.ReLU()
        #self.activation_b = nn.ReLU()
        self.fc_a = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc_b = nn.Linear(hidden_dim//2, output_dim)

    def forward(self, x, h):
        out, h = self.gru(x, h)

        # The following statement is important: we are implementing a S2V model,
        # so we select the latest output.

        out=out[:,-1]

        #out=self.activation_a(out)
        out=self.fc_a(out)
        out=self.activation_a(out)
        out=self.fc_b(out)
        if self.positive_variable:
          #out=torch.exp(out)
          #out=torch.abs(out)
          #out=torch.square(out)
          pass

        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        print("self.parameters() = ", self.parameters())
        print("next(self.parameters()) = ", next(self.parameters()))
        print("weight = next(self.parameters()).data = ", weight)
        print("type of weight = ", type(weight))
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        print("hidden=weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()", weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

class TCNModel(Model):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, positive_variable, kernel_size, num_channels=None, skip_connections = False, drop_prob=0.0):
        super().__init__(input_dim, hidden_dim, output_dim, n_layers, positive_variable, num_channels=None, drop_prob=0.0)
        if num_channels is None:  # if channels is not specified use uniform hidden_size
          num_channels = [hidden_dim] * n_layers
        else:
          self.num_channels = num_channels

        self.tcn = TCN(input_dim, num_channels, kernel_size, dropout=drop_prob, causal=True, use_skip_connections=skip_connections)
        self.fc = nn.Linear(num_channels[-1], output_dim)

    def forward(self, x):
        #TCN expects input shape (batch_size, input size, sequence length)
        out = self.tcn(x.transpose(1, 2))
        out = out[:, :, -1] # it takes the output of the last timestep
        out = self.fc(out)
        return out

In [11]:
def evaluate(model, test_x, test_y, label_scaler, post_process, mm5_array):
    outputs = []
    targets = []

    for i in range(test_x.shape[0]):
        inp = torch.from_numpy(np.expand_dims(test_x[i], axis=0))

        if type(model) == GRUModel:
          out, h = model(inp.to(device).float(), None)
        elif type(model) == TCNModel:
          out = model(inp.to(device).float())

        outputs.append(out[0,0].item())
        targets.append(test_y[i])

        if i%5000==0:
          print(i,'/',test_x.shape[0],sep='')

    predictions_array = label_scaler.inverse_transform(np.array(outputs).reshape(-1, 1))
    target_array = label_scaler.inverse_transform(np.array(targets).reshape(-1, 1))

    if post_process:
      predictions_array = predictions_array * (mm5_array!=0)

    return predictions_array, target_array

In [ ]:
def se_array(x, y):
  return np.square(x-y)

def ae_array(x, y):
  return np.abs(x - y)

def rmse(x,y):
  return np.sqrt(mean_squared_error(x,y))

def mae(x, y):
    return np.mean(np.abs(x-y))

def latex_row(a,b,c,bigger):
  if a>b and bigger or a<b and not bigger:
    return '\\textbf{'+str(round(a, 3))+'}\\\\ '+str(round(b, 3))+'\\\\ '+str(round(c, 2))+'\%'
  else:
    return str(round(a, 3))+'\\\\ \\textbf{'+str(round(b, 3))+'}\\\\ '+str(round(c, 2))+'\%'

In [13]:
def calculate_local_mean(values, window_size=3):
    result = np.zeros_like(values)
    padded = np.pad(values, (window_size, window_size), mode='edge')

    for i in range(len(values)):
        window_start = i
        window_end = i + 2 * window_size + 1
        result[i] = np.mean(padded[window_start:window_end])

    return result

In [14]:
#site3_mm5_mae_1 = []
#site3_gru_mae_1 = []
site3_tcn_mae_1 = []

#site3_mm5_mae_2 = []
#site3_gru_mae_2 = []
site3_tcn_mae_2 = []


#site3_mm5_mae_3 = []
#site3_gru_mae_3 = []
site3_tcn_mae_3 = []


#site3_mm5_r2_1 = []
#site3_gru_r2_1 = []
site3_tcn_r2_1 = []


#site3_mm5_r2_2 = []
#site3_gru_r2_2 = []
site3_tcn_r2_2 = []

#site3_mm5_r2_3 = []
#site3_gru_r2_3 = []
site3_tcn_r2_3 = []

In [ ]:
for LOOK_FORWARD_DAYS in [3]:
  for dataset in DATASETS_TO_PROCESS:
    for variable in VARIABLES_TO_PROCESS:
      print('='*80)
      print('Look Forward Days:', LOOK_FORWARD_DAYS)
      print('Dataset:', DATASET_NAMES[dataset])
      print('Variable:', VARIABLE_NAMES[variable])
      print('_'*80)

      if variable not in HP[dataset].keys():
        continue

      suffix = '_look_forward_days_' + str(LOOK_FORWARD_DAYS) + '_dataset_' + str(dataset) + '_variable_' + str(variable)

      data_root = '/content/drive/MyDrive/Colab Notebooks/datasets/energy/'
      root_tcn = data_root + 'results/'

      report_name_tcn = root_tcn + 'report'+suffix+'.txt'
      report_file_tcn = open(report_name_tcn, "w")

      report_file_tcn.write('Dataset {}, Variable {}\n'.format(str(dataset), VARIABLE_NAMES[variable]))
      report_file_tcn.write('_'*80+'\n')

      # Create dataframe =======================================================

      # ============================================================================
      # DATA LOADING
      # ============================================================================
      # NOTE: Due to confidentiality agreements, the datasets cannot be publicly
      # shared. The code is provided for transparency and reproducibility of the
      # methodology. Researchers with similar datasets can adapt this code for
      # their own data.
      #
      # Expected file structure in Google Drive:
      # MyDrive/Colab Notebooks/datasets/energy/dataset_{0,1}_look_forward_days_{1,2,3}.xlsx
      # ============================================================================

      url = data_root + 'dataset_'+str(dataset)+'_look_forward_days_' + str(LOOK_FORWARD_DAYS)+'.xlsx'
      df = pd.read_excel(url)

      pd.set_option('display.float_format', lambda x: '%.2f' % x)
      #print(df.head)

      df['dayofyear'] = df.apply(lambda x: x['Data_R'].dayofyear,axis=1)
      df['hour'] = df.apply(lambda x: x['Ora_R'].hour,axis=1)
      #df['dayofweek'] = df.apply(lambda x: x['Data_R'].dayofweek,axis=1)
      #df['month'] = df.apply(lambda x: x['Data_R'].month,axis=1)
      #df['minute'] = df.apply(lambda x: x['Ora_R'].minute,axis=1)

      datetime_info = df[['Data_R', 'Ora_R']].copy()

      df = df.drop(["Data_R", "Ora_R", "Data_MM5", "Ora_MM5"], axis=1)
      df = df.replace("None", np.nan)

      if PLOT_DISTRIBUTIONS:
        df[df['GHI_R'].notna()]['GHI_R'].plot.hist(bins=50) # notna() This function takes a scalar or array-like object and indicates whether values
        plt.pause(0.001)                                    # are valid (not missing)
        df[df['Temperatura_R'].notna()]['Temperatura_R'].plot.hist(bins=100)
        plt.pause(0.001)
        df[df['Pressione_R'].notna()]['Pressione_R'].plot.hist(bins=100)
        plt.pause(0.001)
        df[df['Umidità_R'].notna()]['Umidità_R'].plot.hist(bins=100)
        plt.pause(0.001)
        break

      NUMBER_OF_VARIABLES = (len(df.columns)-2)//2
      print('Number of variables:', NUMBER_OF_VARIABLES)

      # Select current hyperparameters =========================================
      HIDDEN_DIM =HP[dataset][variable]['HIDDEN_DIM']
      N_LAYERS = HP[dataset][variable]['N_LAYERS']
      BATCH_SIZE = HP[dataset][variable]['BATCH_SIZE']
      LEARNING_RATE = HP[dataset][variable]['LEARNING_RATE']
      TEST_PLOT_START = HP[dataset][variable]['TEST_PLOT_START'][LOOK_FORWARD_DAYS-1]
      TEST_PLOT_STOP = TEST_PLOT_START+144*DAYS_TO_PLOT
      EPOCHS = HP[dataset][variable]['EPOCHS']
      NUM_CHANNELS = HP[dataset][variable]['NUM_CHANNELS']
      KERNEL_SIZE = HP[dataset][variable]['KERNEL_SIZE']
      SKIP_CONNECTIONS = HP[dataset][variable]['SKIP_CONNECTIONS']



      for hp in HP[dataset][variable].keys():
        print(hp,': ', HP[dataset][variable][hp],sep='')
        pass

      # Create scalers =========================================================
      data=df.astype('float32').to_numpy()
      print(df.head())

      if STANDARD_SCALERS:
        scaler = StandardScaler()
        label_scaler = StandardScaler()
        mm5_scaler = StandardScaler()
      else:
        scaler = MinMaxScaler()
        label_scaler = MinMaxScaler()
        mm5_scaler = MinMaxScaler()

      if dataset==0 or variable==0:
        label_scaler.fit(data[:,variable].reshape(-1,1))
        mm5_scaler.fit(data[:,variable+NUMBER_OF_VARIABLES].reshape(-1,1))
      else:
        label_scaler.fit(data[:,variable-1].reshape(-1,1))
        mm5_scaler.fit(data[:,variable+NUMBER_OF_VARIABLES-1].reshape(-1,1))

      data = scaler.fit_transform(data)

      if DEBUG:
        print(scaler.data_min_)
        print(scaler.data_max_)
        print(label_scaler.data_min_)
        print(label_scaler.data_max_)
        print(mm5_scaler.data_min_)
        print(mm5_scaler.data_max_)

      # Define lookback period and split inputs/labels =========================
      look_back_steps = 144 * LOOK_BACK_DAYS
      look_forward_steps = 144 * LOOK_FORWARD_DAYS

      #inputs = np.zeros((len(data)-look_back_steps-look_forward_steps+1,look_back_steps,df.shape[1]))
      inputs = np.zeros((len(data)-look_back_steps-look_forward_steps+1,look_back_steps,NUMBER_OF_VARIABLES+2)) # MM5

      local_means = np.zeros(len(data)-look_back_steps-look_forward_steps+1)

      labels = np.zeros(len(data)-look_back_steps-look_forward_steps+1)
      dates = np.zeros(len(data)-look_back_steps-look_forward_steps+1, dtype='object')
      hours = np.zeros(len(data)-look_back_steps-look_forward_steps+1, dtype='object')

      if DEBUG:
        print(data.shape)
        print(inputs.shape)
        print(labels.shape)

      mask=np.full(inputs.shape[0], True)

      # ONLY MM5 DATA:

      # The input is a sequence of vectors containing the MM5 values from time
      # 'T-look_back_steps+1' to time 'T' (where T is future instant of time the
      # prediction is referred to)
      # Try look_back_steps=look_forward_steps=1

      # The models takes in input only MM5 values that cannot be null
      # That's why some data rows previously deleted now are included and the
      # MM5 performance metrics are slightly different.

      for i in range(look_back_steps, len(data) - look_forward_steps + 1):

        index = i-look_back_steps

        #inputs[index] = data[index : i]
        inputs[index] = data[index : i, NUMBER_OF_VARIABLES:] # MM5

        if dataset==0 or variable==0:
          labels[index] = data[i + look_forward_steps - 1, variable]
        else:
          labels[index] = data[i + look_forward_steps - 1, variable-1]

        dates[index] = datetime_info['Data_R'][i + look_forward_steps - 1]
        hours[index] = datetime_info['Ora_R'][i + look_forward_steps - 1]

        if np.isnan(inputs[index]).any() or np.isnan(labels[index]).any():
          #print('input:',inputs[index])
          #print('label:',labels[index])
          mask[index] = False

        if i%10000==0 and DEBUG:
          print(i)
          print('-'*80)
          print(inputs[index])
          print(labels[index])

      if DEBUG or True:
        print('BEFORE')
        print(inputs.shape)
        print(labels.shape)
        print('nan inputs:', np.isnan(inputs).any())
        print('nan labels:', np.isnan(labels).any())
        print('Number of nan tuples:', np.sum(np.logical_not(mask)))

      inputs=inputs[mask]
      labels=labels[mask]
      dates = dates[mask]
      hours = hours[mask]

      if DEBUG or True:
        print('AFTER')
        print(inputs.shape)
        print(labels.shape)
        print('nan inputs:', np.isnan(inputs).any())
        print('nan labels:', np.isnan(labels).any())

      #inputs = inputs.reshape(-1, look_back_steps, df.shape[1])
      inputs = inputs.reshape(-1, look_back_steps, NUMBER_OF_VARIABLES +2) # MM5
      labels = labels.reshape(-1, 1)

      local_means = calculate_local_mean(labels.flatten())
      local_means = local_means.reshape(-1, 1)

      # Split data into training set, validation set and test set
      validation_portion = int(VALIDATION_FRACTION*len(inputs))
      test_portion = int(TEST_FRACTION*len(inputs))
      validation_test_portion = validation_portion + test_portion

      train_x = inputs[:-validation_test_portion]
      train_y = labels[:-validation_test_portion]
      train_local_means = local_means[:-validation_test_portion]

      validation_x = inputs[-validation_test_portion:-test_portion]
      validation_y = labels[-validation_test_portion:-test_portion]
      validation_local_means = local_means[-validation_test_portion:-test_portion]

      test_x = inputs[-test_portion:]
      test_y = labels[-test_portion:]
      test_dates = dates[-test_portion:]
      test_hours = hours[-test_portion:]

      # Create dataloaders =====================================================
      train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y), torch.from_numpy(train_local_means))
      train_loader = DataLoader(train_data, shuffle=SHUFFLE, batch_size=BATCH_SIZE, drop_last=True)

      validation_data = TensorDataset(torch.from_numpy(validation_x), torch.from_numpy(validation_y), torch.from_numpy(validation_local_means))
      validation_loader = DataLoader(validation_data, shuffle=SHUFFLE, batch_size=BATCH_SIZE, drop_last=True)

      train_loss_list_gru=None
      train_loss_list_tcn=None
      validation_loss_list_gru=None
      validation_loss_list_tcn=None

      # Create the model =======================================================
      model_name_tcn = root_tcn + 'model' + suffix + '.zip'

      if Path(model_name_tcn).is_file() and not FORCE_TRAINING:
          input_dim = next(iter(train_loader))[0].shape[2]
          model_tcn = TCNModel(input_dim, hidden_dim=HIDDEN_DIM, output_dim=1, n_layers=N_LAYERS, positive_variable=(variable in POSITIVE_VARIABLES), kernel_size=KERNEL_SIZE, num_channels=NUM_CHANNELS, skip_connections=SKIP_CONNECTIONS)
          model_tcn.load_state_dict(torch.load(model_name_tcn))
          model_tcn.to(device)
      else:
          print("training start")
          model_tcn, train_loss_list_tcn, validation_loss_list_tcn = train(model_name_tcn, train_loader, validation_loader, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE, hidden_dim=HIDDEN_DIM, n_layers=N_LAYERS, positive_variable=(variable in POSITIVE_VARIABLES), patience=PATIENCE, epochs=EPOCHS, num_channels=NUM_CHANNELS, kernel_size=KERNEL_SIZE, skip_connections=SKIP_CONNECTIONS)

      model_tcn.to(device)
      model_tcn.eval()



      # Evaluation =============================================================

      if dataset==0 or variable==0:
        #mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable + NUMBER_OF_VARIABLES].reshape(-1,1))
        mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable].reshape(-1,1)) # MM5
      else:
        #mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable + NUMBER_OF_VARIABLES-1].reshape(-1,1))
         mm5_array = mm5_scaler.inverse_transform(test_x[:, -1, variable].reshape(-1,1)) # MM5

      predictions_array_tcn, targets_array = evaluate(model_tcn, test_x, test_y, label_scaler, (variable in POSITIVE_VARIABLES) and POST_PROCESS, mm5_array)


      filename_xlsx = root_tcn + 'arrays'+suffix+'.xlsx'
      formatted_dates = [d.strftime('%d/%m/%Y') for d in test_dates]
      df_results = pd.DataFrame({
                      'Datae': formatted_dates,
                      'Hour': test_hours,
                      'Real': targets_array.flatten(),
                      'MM5': mm5_array.flatten(),
                      'Prediction': predictions_array_tcn.flatten()
                    })

      df_results.to_excel(filename_xlsx, index=False)

      print('_'*80)



      # MAE ===================================================================
      prediction_mae_tcn = mae(predictions_array_tcn, targets_array)
      mm5_mae = mae(mm5_array, targets_array)
      delta_mae_tcn = round((prediction_mae_tcn-mm5_mae)/mm5_mae*100,2)*np.sign(mm5_mae)

      if dataset == 0:
        if LOOK_FORWARD_DAYS == 1:
          site3_tcn_mae_1.append(prediction_mae_tcn)
        elif LOOK_FORWARD_DAYS == 2:
          site3_tcn_mae_2.append(prediction_mae_tcn)
        elif LOOK_FORWARD_DAYS == 3:
          site3_tcn_mae_3.append(prediction_mae_tcn)

      print('MM5 mae: {:.3f}'.format(mm5_mae))
      print('Prediction mae tcn: {:.3f}'.format(prediction_mae_tcn))
      print('Delta mae tcn: {:.3f} %'.format(delta_mae_tcn))


      report_file_tcn.write('MM5 mae: {:.3f}\n'.format(mm5_mae))
      report_file_tcn.write('Prediction mae tcn: {:.3f}'.format(prediction_mae_tcn))
      report_file_tcn.write('Delta mae tcn: {:.3f} %'.format(delta_mae_tcn))

      print('_'*80)

      number_of_outliers = int(len(targets_array)*OUTLIERS)
      print('number_of_outliers:',number_of_outliers)


      # MAE Outliers ===========================================================
      prediction_ae_array_tcn=ae_array(predictions_array_tcn, targets_array).T[0]
      mm5_ae_array=ae_array(mm5_array, targets_array).T[0]

      indices_of_prediction_outliers_tcn = np.argpartition(prediction_ae_array_tcn, -number_of_outliers)[-number_of_outliers:]
      indices_of_mm5_outliers = np.argpartition(mm5_ae_array, -number_of_outliers)[-number_of_outliers:]

      indices_of_outliers_tcn = list(set(indices_of_prediction_outliers_tcn) | set(indices_of_mm5_outliers))

      print(len(predictions_array_tcn))
      print(len(indices_of_outliers_tcn))

      outliers_prediction_mae_tcn = mae(predictions_array_tcn[indices_of_outliers_tcn], targets_array[indices_of_outliers_tcn])
      outliers_mm5_mae = mae(mm5_array[indices_of_outliers_tcn], targets_array[indices_of_outliers_tcn])
      outliers_delta_mae_tcn = round((outliers_prediction_mae_tcn-outliers_mm5_mae)/outliers_mm5_mae*100,2)*np.sign(outliers_mm5_mae)

      print('Outliers MM5 mae: {:.3f}'.format(outliers_mm5_mae))
      print('Outliers Prediction tcn mae: {:.3f}'.format(outliers_prediction_mae_tcn))
      print('Outliers Delta tcn mae: {:.3f} %'.format(outliers_delta_mae_tcn))

      report_file_tcn.write('Outliers MM5 mae: {:.5f}\n'.format(outliers_mm5_mae))
      report_file_tcn.write('Outliers Prediction tcn mae: {:.3f}\n'.format(outliers_prediction_mae_tcn))
      report_file_tcn.write('Outliers Delta tcn mae: {:.3f} %\n\n'.format(outliers_delta_mae_tcn))
      print('_'*80)


      # R^2 ====================================================================
      r2_prediction_tcn = r2_score(targets_array, predictions_array_tcn)
      r2_mm5 = r2_score(targets_array, mm5_array)
      delta_r2_tcn = round((r2_prediction_tcn-r2_mm5)/r2_mm5*100,2)*np.sign(r2_mm5)

      if dataset == 0:
        if LOOK_FORWARD_DAYS == 1:
          site3_tcn_r2_1.append(r2_prediction_tcn)
        elif LOOK_FORWARD_DAYS == 2:
          site3_tcn_r2_2.append(r2_prediction_tcn)
        elif LOOK_FORWARD_DAYS == 3:
          site3_tcn_r2_3.append(r2_prediction_tcn)

      print('MM5 R²: {:.3f}'.format(r2_mm5))
      print('Prediction tcn R²: {:.3f}'.format(r2_prediction_tcn))
      print('Delta tcn R²: {:.3f} %'.format(delta_r2_tcn))


      report_file_tcn.write('MM5 R²: {:.3f}\n'.format(r2_mm5))
      report_file_tcn.write('Prediction tcn R²: {:.3f}'.format(r2_prediction_tcn))
      report_file_tcn.write('Delta tcn R²: {:.3f} %'.format(delta_r2_tcn))

      print('_'*80)


      # R^2 Outliers ===========================================================
      prediction_se_array_tcn=se_array(predictions_array_tcn, targets_array).T[0]
      mm5_se_array=se_array(mm5_array, targets_array).T[0]

      indices_of_prediction_outliers_tcn = np.argpartition(prediction_se_array_tcn, -number_of_outliers)[-number_of_outliers:]
      indices_of_mm5_outliers = np.argpartition(mm5_se_array, -number_of_outliers)[-number_of_outliers:]

      indices_of_outliers_tcn = list(set(indices_of_prediction_outliers_tcn) | set(indices_of_mm5_outliers))

      outliers_r2_prediction_tcn = r2_score(predictions_array_tcn[indices_of_outliers_tcn], targets_array[indices_of_outliers_tcn])
      outliers_r2_mm5 = r2_score(mm5_array[indices_of_outliers_tcn], targets_array[indices_of_outliers_tcn])
      outliers_delta_r2_tcn = round((outliers_r2_prediction_tcn-outliers_r2_mm5)/outliers_r2_mm5*100,2)*np.sign(outliers_r2_mm5)

      print('Outliers MM5 R²: {:.3f}'.format(outliers_r2_mm5))
      print('Outliers Prediction tcn R²: {:.3f}'.format(outliers_r2_prediction_tcn))
      print('Outliers Delta tcn R²: {:.3f} %'.format(outliers_delta_r2_tcn))

      report_file_tcn.write('Outliers MM5 R²: {:.3f}\n'.format(outliers_r2_mm5))
      report_file_tcn.write('Outliers Prediction tcn R²: {:.3f}'.format(outliers_r2_prediction_tcn))
      report_file_tcn.write('Outliers Delta tcn R²: {:.3f} %'.format(outliers_delta_r2_tcn))

      report_file_tcn.close()

      print('_'*80)

      # Plot ===================================================================
      TEST_PLOT_START = 180
      TEST_PLOT_STOP = TEST_PLOT_START + 144*3
      days=(TEST_PLOT_STOP-TEST_PLOT_START)//144
      print('Plotting results over ', days,' days',sep='')

      shift=0#(3-LOOK_FORWARD_DAYS)*260

      plt.figure(figsize=(20,10))
      plt.subplot(2,2,1)
      plt.plot(targets_array[TEST_PLOT_START:TEST_PLOT_STOP], color="g", label="Real", linewidth=0.7)
      plt.plot(mm5_array[TEST_PLOT_START:TEST_PLOT_STOP], color="b", label="MM5", linestyle="dashed", linewidth=0.7)
      plt.plot(predictions_array_tcn[TEST_PLOT_START:TEST_PLOT_STOP], color="r", label="Predicted tcn", linestyle="dashdot", linewidth=0.7)
      plt.ylabel(VARIABLE_NAMES[variable]+' ['+UNITS_OF_MEASUREMENT[variable]+']')
      plt.xlabel('Time')
      plt.legend()


      # Save the figure ========================================================
      figure_name = root_tcn + 'figure'+suffix+'.png'
      plt.savefig(figure_name, dpi=(250), bbox_inches='tight')


    if PLOT_DISTRIBUTIONS:
      break

  if PLOT_DISTRIBUTIONS:
    break

In [ ]:
if not PLOT_DISTRIBUTIONS:
    # Plot =====================================================================
    TEST_PLOT_START = 180
    TEST_PLOT_STOP = TEST_PLOT_START + 144*3

    days=(TEST_PLOT_STOP-TEST_PLOT_START)//144
    print('Plotting results over ', days,' days',sep='')

    plt.figure(figsize=(20,10))
    plt.subplot(2,2,1)
    plt.plot(targets_array[TEST_PLOT_START:TEST_PLOT_STOP], color="g", label="Real", linewidth=0.7)
    plt.plot(mm5_array[TEST_PLOT_START:TEST_PLOT_STOP], color="b", label="MM5", linestyle='dashed', linewidth=0.7)
    plt.plot(predictions_array_tcn[TEST_PLOT_START:TEST_PLOT_STOP], label="TCN Predicted", linestyle='dashdot', linewidth=0.7)
    plt.ylabel(VARIABLE_NAMES[variable])
    plt.legend()